In [1]:
import causaldag as cd
from causaldag.structure_learning import gsp
from causaldag.utils.ci_tests import gauss_ci_test, MemoizedCI_Tester, gauss_ci_suffstat
from causaldag.utils.ci_tests import hsic_test, dsep_test
import numpy as np
from pprint import pprint
import random

# GSP on known graph

## Set up graph, samples, and correlation matrix

Create the graph $0 \rightarrow 1 \leftarrow 2$

In [2]:
dag = cd.DAG(arcs={(0, 1), (2, 1)})

Turn the graph into GaussDAG, which will allow us to sample from it. Use random edge weights to avoid faithfulness violation. By default, edge weights are sampled uniformly from $\pm[.25, 1]$.

In [3]:
gdag = cd.rand.rand_weights(dag)

Take $n$ samples

In [4]:
nsamples = 500
np.random.seed(1729)
random.seed(1729)
samples = gdag.sample(nsamples)

Form the sufficient statistics dictionary for the CI test.

*The gauss_ci test requires a correlation matrix and the number of samples. Adding the inverse correlation matrix and the partial correlation matrix to the dictionary of sufficient statistics is essential for improved speed when the number of nodes is large. gauss_ci_suffstat is a helper function that will do this work for you.*

In [5]:
suffstat = gauss_ci_suffstat(samples)

## Run GSP and explore results

In [6]:
nnodes = 3
nodes = set(range(nnodes))
np.random.seed(1729)
random.seed(1729)

gauss_ci_tester = MemoizedCI_Tester(gauss_ci_test, suffstat, alpha=.05)
est_dag_gauss, summaries_gauss = gsp(nodes, gauss_ci_tester, depth=4, nruns=2)

oracle_ci_tester = MemoizedCI_Tester(dsep_test, dag)
est_dag_oracle, summaries_oracle = gsp(nodes, oracle_ci_tester, depth=4, nruns=2)

Print the result. The convention for displaying a DAG as a string follows pcalg/bnlearn in R: [i|j,k,l] means that j,k, and l are parents of i.

In [7]:
print(est_dag_gauss)

[2][0][1|0,2]


In [8]:
print(est_dag_oracle)

[2][0][1|0,2]


GSP returns the smallest DAG found over the course of multiple runs of the algorithm. `summaries` is a list containing details about each run. 

Each summary run's summary lists the DAGs in the order they were visited, their sparsity, and the search depth of the depth-first search procedure.

In [9]:
pprint(summaries_gauss[0])  # in this run, the starting DAG had no covered edges

[{'dag': [2][0][1|0,2], 'depth': 0, 'num_arcs': 2}]


In [10]:
pprint(summaries_gauss[1])  # this run is less trivial

[{'dag': [0][1|0][2|1], 'depth': 0, 'num_arcs': 3},
 {'dag': [0][1|0][2|1], 'depth': 1, 'num_arcs': 3},
 {'dag': [0][1|0][2|1], 'depth': 2, 'num_arcs': 3},
 {'dag': [0][1|0][2|1], 'depth': 3, 'num_arcs': 3},
 {'dag': [0][1|0][2|1], 'depth': 0, 'num_arcs': 2},
 {'dag': [0][1|0][2|1], 'depth': 1, 'num_arcs': 2},
 {'dag': [0][1|0][2|1], 'depth': 2, 'num_arcs': 2},
 {'dag': [1][2|1][0|1], 'depth': 1, 'num_arcs': 2},
 {'dag': [2][1|2][0|1], 'depth': 0, 'num_arcs': 2}]


Use the non-parametric HSIC test as the CI test. The sufficient statistic for this test is simply the data itself (note: in the future, this should be a dictionary for the sake of consistency).

In [11]:
np.random.seed(1729)
random.seed(1729)
hsic_tester = MemoizedCI_Tester(hsic_test, samples)
est_dag_hsic, summaries_hsic = gsp(nodes, hsic_tester)

In [12]:
print(est_dag_hsic)

[2][0][1|0,2]
